## 7. Functions

函数存在一些技巧，主要是用来封装和装饰（decorate）。

此外还有一大块是 回调函数。 放到 异步处理去。

几大块

1. 传参，未知参数，键值参数，默认参数
2. 封装和装饰
3. 异步回调
4. 闭包

### 7.1  任意参数函数

`*args, **kwargs`

In [15]:
# Examples of *args and **kwargs functions

def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

print(avg(1, 2))
print(avg(1,2,3,4))

import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                  name=name,
                  attrs=attr_str,
                  value=html.escape(value))
    return element

# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
print(make_element('item', 'Albatross', size='large', quantity=6))
print(make_element('p','<spam>'))


1.5
2.5
<item size="large" quantity="6">Albatross</item>
<p>&lt;spam&gt;</p>


### 7.2 某些参数必须指明key （keyword arguments）

这是为了避免传参时，参数顺序错误导致的错误。

In [5]:
# examples of keyword-only argument functions

# A simple keyword-only argument
def recv(maxsize, *, block=True):
    print(maxsize, block)

recv(8192, block=False)        # Works
try:
    recv(8192, False)          # Fails
except TypeError as e:
    print(e)


8192 False
recv() takes 1 positional argument but 2 were given


In [6]:

# Adding keyword-only args to *args functions
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

print(minimum(1, 5, 2, -5, 10))
print(minimum(1, 5, 2, -5, 10, clip=0))


-5
0


### 7.3 函数参数类型声明（其实是一种标注）

In [9]:
def add(x:int, y:int) -> int:
    """add 2 nums
    """
    return x + y

print(help(add))
print(add.__annotations__)

Help on function add in module __main__:

add(x:int, y:int) -> int
    add 2 nums

None
{'x': <class 'int'>, 'y': <class 'int'>, 'return': <class 'int'>}


### 7.4 一个函数返回多个值

这个很简单， 用','分隔开就行了，返回的相当于是一个元组

### 7.5 默认参数

这个也简单，但是会有一些小坑，比如默认参数值不能设置字典、列表之类的。

### 7.6 匿名函数和Inline函数


lambda函数。 也有一些小坑。 例如，lambda函数里面使用的外部变量，值是由调用时的外部变量值决定的。

### 7.7 简化函数参数

partial ，简化参数。 

感觉其实就是对原函数进行了一次封装。

In [11]:
from functools import partial

points = [ (1, 2), (3, 4), (5, 6), (7, 7) ]

import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

pt = (4,3)
# 距离 pt 最近的点
points.sort(key=partial(distance, pt))
print(points)


[(3, 4), (1, 2), (5, 6), (7, 7)]


In [13]:
import functools

def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

# A sample function
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial

    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')

    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()


DEBUG:test:Got: 7


In [14]:
# Using partial to supply extra arguments to a class constructor
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    # ack is added keyword-only argument. *args, **kwargs are
    # any normal parameters supplied (which are passed on)
    def __init__(self, *args, ack, **kwargs):
        self.ack = ack
        super().__init__(*args, **kwargs)
    def handle(self):
        for line in self.rfile:
            self.wfile.write(self.ack + line)

if __name__ == '__main__':
    from functools import partial
    serv = TCPServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))
    print('Echo server running on port 15000')
    serv.serve_forever()


Echo server running on port 15000


KeyboardInterrupt: 

### 7.9 封装一个类的方法

其实也是简化封装。

### 7.12 Closure 概念

我的理解就是函数里面定义函数。闭包。

注意下面例子里面的nonlocal。

有时候 闭包可以替代一些类， 会比类运行的更快一些

In [16]:
# Example of accessing variables inside a closure

def sample():
    n = 0           
    # Closure function
    def func():
        print('n=', n)
    
    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

if __name__ == '__main__':
    f = sample()
    f()
    n= 0
    f.set_n(10)
    f()
    print(f.get_n())


n= 0
n= 10
10
